In [ ]:
import os
import pickle

import pandas as pd

from chtc.preprocess_extraction_direct import hard_reset, push, Triplets

os.chdir("..")

In [ ]:
# Dump the data
# df = export("triplets")
# df.to_parquet("backup.parquet.gzip", compression="gzip")

df = pd.read_parquet("backup.parquet.gzip")
len(df)

In [ ]:
# Purge the database and recreate with new schema
hard_reset()

In [ ]:
# Load the master ids list
all_ids_pickle = "geoarchive_paragraph_ids.pkl"
with open(all_ids_pickle, "rb") as f:
    all_ids = pickle.load(f)

# Split the ids into jobs
n_jobs = (len(all_ids) // 2000) + 1
job_ids = range(n_jobs)

# Create a mapping from id to job_id
id2job = {}
for job_id in job_ids:
    this_job_ids = all_ids[job_id * 2000 : (job_id + 1) * 2000]
    id2job.update({i: job_id for i in this_job_ids})

In [ ]:
# Append the job_id to the data
triplets = []

for row in df.iterrows():
    data_pack = {
        "id": row[1]["id"],
        "job_id": id2job[row[1]["id"]],
        "hashed_text": row[1]["hashed_text"],
        "paper_id": row[1]["paper_id"],
        "triplets": row[1]["triplets"],
    }
    triplets.append(Triplets(**data_pack))

In [ ]:
from tqdm.auto import tqdm

batch_size = 100
for i in tqdm(range(0, len(triplets), batch_size)):
    push(triplets[i : i + batch_size])